# Trabajo práctico 1. Introducción al Análisis de Datos

## ¿Qué tan cálida era Europa en los últimos 500 años?

**Problema.** Intentaremos responder las siguientes preguntas:

1. ¿Cuál era la temperatura extrema en los últimos 500 años en Europa?

2. ¿Cómo evolucionó la temperatura?

3. ¿Cómo era en un determinado punto del tiempo (fecha o aproximación)?

## 1. Obtención de datos

Descargar los [datos](https://drive.google.com/file/d/1eEZCnCuGKU2LFKrWaC0Tr8LwgxrZLqCk/view?usp=sharing) y descomprimir.

In [ ]:
# Cambiar a lo que corresponda
DATASET_PATH = "./data"

In [ ]:
!ls {DATASET_PATH}

In [ ]:
!cat {DATASET_PATH}/europe-seasonal.txt

## 2. Análisis exploratorio inicial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
# Observaciones 
# 1: Year
# 2: Winter (December-January-February) averaged absolute surface air temperature, �C 
# 3: Spring (March-April-May) averaged absolute surface air temperature, �C 
# 4: Summer (June-July-August) averaged absolute surface air temperature, �C 
# 5: Autumn (September-October-November) averaged absolute surface air temperature, �C 
# 6: Annual averaged absolute surface air temperature, �C 
df = pd.read_csv(DATASET_PATH+"/europe-seasonal.dat",
                 names=["year","winter","spring","summer","autumn","annual"],
                 delim_whitespace=True 
)
df.head(5)

Obtener alguna información básica sobre la temperatura en Europa en los últimos 500 años.

In [ ]:
winter_min = 0 # COMPLETAR
year_winter_min = 0 # COMPLETAR
print("La temperatura promedio más baja en invierno es: {:+7.3f}C".format(winter_min))
print("Esto ocurrió en el año: {}".format(year_winter_min))

In [ ]:
winter_max = 0 # COMPLETAR
year_winter_max = 0 # COMPLETAR
print("La temperatura promedio más alta en invierno es: {:+7.3f}C".format(winter_max))
print("Esto ocurrió en el año: {}".format(year_winter_max))

In [ ]:
summer_min = 0 # COMPLETAR
year_summer_min = 0 # COMPLETAR
print("La temperatura promedio más baja en verano es: {:+7.3f}C".format(summer_min))
print("Esto ocurrió en el año: {}".format(year_summer_min))

In [ ]:
summer_max = 0 # COMPLETAR
year_summer_max = 0 # COMPLETAR
print("La temperatura promedio más alta en invierno es: {:+7.3f}C".format(summer_max))
print("Esto ocurrió en el año: {}".format(year_summer_max))

In [ ]:
min_annual_temperature = 0 # COMPLETAR
min_annual_temperature_year = 0  # COMPLETAR
max_annual_temperature = 0  # COMPLETAR
max_annual_temperature_year = 0 # COMPLETAR
print("La temperatura anual promedio varió entre {:+.3f}C en el año {} a {:+.3f}C en el año {}.".format( 
    min_annual_temperature, min_annual_temperature_year,
    max_annual_temperature, max_annual_temperature_year
))

Nota. De ahora en más, para obtener estadísticas generales usaremos la función *describe()*.

In [ ]:
df.describe()

## 3. Gráfico de tendencia de la temperatura

El análisis anterior nos dió una primera impresión de los valores de la temperatura en la segunda mitad del pasado mileno.
No obstante, para tener una visión más completa, es conveniente recurrir a un gráfico de tendencia como el que se muestra a continuación:

<center>
    <img src="assets/tendencia_temperatura_completo.png"/>
</center>

Escribir una función *plot_temperature_trends* que permita graficar la tendencia de las series correspondientes a las cuatro estaciones para los años indicados. Si no se especifica ningún año de inicio y/o fin utilizar el mínimo y máximo de la serie respectivamente.

In [ ]:
def plot_temperature_trends(df,filename,start_year=None,end_year=None):
    """Realiza un gráfico de tendencia y lo almacena en un archivo.

    :param df: Dataframe con valores de temperaturas anuales. 
               Debe contener las columnas 'year' y 'autumn','winter','spring','summer', 'annual'.               
    :type pd.Dataframe        
    :param filename: Nombre del archivo de salida
    :type string        
    :param start_year: Año inicial. Si se omite, es el primero de la serie.
    :type int, optional
    :param end_year: Año final. Si se omite, es el último de la serie.
    :type int, optional        
    """
    # COMPLETAR
    # ...
    
plot_temperature_trends(df,filename="tendencia_temperatura_completo.png",start_year=None,end_year=None)

Del gráfico anterior, podemos realizar las siguientes observaciones:

1. Los inviernos siempre son más fríos que la primavera, que a su vez son más frías que los otoños, que a su vez son más frías que el otoño.
2. La temperatura promedio varía más en invierno que en otras estaciones.
3. Las primaveras varían más desde alrededor del comienzo del siglo 19.

Lo que no podemos observar con claridad son tendencias. Podríamos responder, por ejemplo, ¿si está incrementándose la temperatura?
Los números sugieren que la temperatura está incrementándose porque las temperaturas en invierno, verano y promedio ocurren en los años recientes. Sin embargo, estos valors son sólo extremos que pueden o no estar correlacionados con un comportamiento general de la temperatura.

Para observar mejor este fenómeno, podemos utilizar [suavizado](http://en.wikipedia.org/wiki/Smoothing).

Hay diferentes algoritmos de suavizado. Aquí usaremos el [promedio móvil](http://en.wikipedia.org/wiki/Moving_average) en su forma más simple. 

Si la temperatura para una cierta estación en el año $y$ está dada por la variable $L_y$, entonces crearemos nuevas variables:

$$
L'_y := \frac{1}{2r+1} \sum_{k=y-r}^{y+r} L_k,
$$

Por ejemplo, $L'_y$ es el valor promedio entre las temperaturas del año $y-r$ hasta (e incluyendo) $y+r$, donde $r$ (el radio) es un parámetro dado. Cuanto mayor $r$, más mayor el grado de suavizado.

¿Cómo calcular el promedio móvil?

Comencemos por plantearlo para un elemento, el $i$-ésimo. Esto significa computar la suma $\sum_{k=i-r}^{i+r} L_k$ y dividirla por $2r+1$. Esto significa que debemos:

* obtener una parte de la serie: `L[i-r:i+r+1]` (el `+1` está aquí porque el límite izquierdo no se incluye como parte de la nueva lista),

* calcular la suma: $\sum_{k=i-r}^{i+r} L_k =$`sum(L[i-r:i+r+1])`,

* dividir por `2*r+1`: $\frac{1}{2r+1} \sum_{k=i-r}^{i+r} L_k =$ `sum(L[i-r:i+r+1]) / (2*r+1)`.

Pista: en python, se puede repetir el procedimiento anterior para todos los índices posibles `i` utilizando list comprehension.

In [ ]:
def smooth(L,r):
    """ Dada una serie, devuelver una nueva serie con el suavizado por promedio móvil para radio r.    
    :param L: Serie
    :type: pd.Series
    :param r: grado de suavizado.
    :type: int
    :return: Serie suavizada
    :rtype: pd.Series
    """
    # COMPLETAR
    new_series_values = [] # ...
    return pd.Series(new_series_values)

series = df.autumn

Visualización con distintos niveles de suavizado.

In [ ]:
smoothing_params = [2,4,32,64]
fig,axes=plt.subplots(len(smoothing_params)+1,1,figsize=(22,4*len(smoothing_params)))
axes[0].plot(df.year,series)
axes[0].set_title("Sin suavizado")

for i,r in enumerate(smoothing_params):
    axes[1+i].plot(df.year.iloc[r:-r],smooth(series,r),lw=4)
    axes[1+i].set_title("Con suavizado r={}".format(r))

In [ ]:
def plot_temperature_trends_smooth(df,filename,r=1,start_year=None,end_year=None):
    """Realiza un gráfico de tendencia y lo almacena en un archivo.

    :param df: Dataframe con valores de temperaturas anuales. 
               Debe contener las columnas 'year' y 'autumn','winter','spring','summer', 'annual'.               
    :type pd.Dataframe        
    :param filename: Nombre del archivo de salida
    :type string        
    :param start_year: Año inicial. Si se omite, es el primero de la serie.
    :type int, optional
    :param end_year: Año final. Si se omite, es el último de la serie.
    :type int, optional        
    """
    # COMPLETAR
    # ...
    
plot_temperature_trends_smooth(df,filename="tendencia_temperatura_suavizado.png",
                               r=17,start_year=None,end_year=None)

Opcional: es posible mejorar el gráfico anterior mostrando los valores con distinto suavizado con menor opacidad, como se muestra en la figura.

<center>
<img src="assets/tendencia_temperatura_suavizado2.png"/>
</center>

Opcional: Implementar la función plot_temperature_trends_smooth2 que realiza un gráfico similar al anterior para una lista de valores de suavizado, por ejemplo [ 0, 1, 3, 11, 17 ].
Pista: usar el atributo alpha para controlar la opacidad de los gráficos y label, cuidando de establecer en None todas las etiquetas excepto las de menor opacidad.

In [ ]:
def plot_temperature_trends_smooth2(df,filename,start_year=None,end_year=None):
    """Realiza un gráfico de tendencia y lo almacena en un archivo.

    :param df: Dataframe con valores de temperaturas anuales. 
               Debe contener las columnas 'year' y 'autumn','winter','spring','summer', 'annual'.               
    :type pd.Dataframe        
    :param filename: Nombre del archivo de salida
    :type string        
    :param start_year: Año inicial. Si se omite, es el primero de la serie.
    :type int, optional
    :param end_year: Año final. Si se omite, es el último de la serie.
    :type int, optional        
    """
    # COMPLETAR
    # ...
    
plot_temperature_trends_smooth2(df,filename="tendencia_temperatura_suavizado2.png", start_year=None,end_year=None)

## 4. Mapa de temperaturas

Hasta ahora hemos visto las temperaturas promedio, pero disponemos también de información de las temperaturas para cada punto del mapa.

Para esto trabajaremos con el archivos TT_Europe_1500_2002_New.GDX, que se describe en Readme_TT_1500_2002.txt.

In [ ]:
!ls {DATASET_PATH}

In [ ]:
!cat {DATASET_PATH}/Readme_TT_1500_2002.txt

Dado que tenemos datos geográficos de Europa para los últimos 500 años, una cosa que podemos hacer es mostrar un mapa de temperaturas para cualquier año, como se muestra a continuación:

<center>
<img src=""/>
</center>


In [ ]:
!ls -lh {DATASET_PATH}/TT_Europe_1500_2002_New.GDX

Algunas observaciones:

- **Tamaño del archivo de entrada**: En este caso el archivo puede ser demasiado grande para procesar con Pandas, así que adoptaremos otra estrategia. Como sólo estamos interesados en los valores correspondientes a un año, escribiremos una función *get_temperatures()* que obtenga los valores para cada celda para una fecha dada.
- El formato de los datos es de una grilla que se obtuvo de aplicar una proyección (llamada [equirectangular](http://en.wikipedia.org/wiki/Equirectangular_projection). Nota: el área cubierta es 25W-40E y 35N-70N, en lugar de 30N-70N, como dice el README.

In [ ]:
code2season = {
    13: "Invierno",
    14: "Primavera",
    15: "Verano",
    16: "Otoño"
}

# COMPLETAR, diccionario que debe devolver la inversa: 
# {'Invierno': 13, 'Primavera': 14, 'Verano': 15, 'Otoño': 16}
season2code = {} # COMPLETAR

In [ ]:
def get_temperatures(gdx_filename, year, season ):        
    global code2season, season2code
    # String para filtrar, construida como indica el README.
    year_season_filter = "{:04d}{:02d}".format(year, season2code[season])

    data = None
    with open(gdx_filename, mode="rt", encoding="utf8") as f:
        for line in f:
            # "yyyyss"
            if line[:6] == year_season_filter:
                data = [float(x) for x in line[6:].strip().split()]
                break
    return data

data = get_temperatures(DATASET_PATH+"/TT_Europe_1500_2002_New.GDX",year = 1700,season = "Otoño")
len(data)

Notar que hemos obtenido una lista de 9100 puntos, pero necesitamos una grilla. 
Convertiremos la lista a una array de 70x130 de Numpy.

In [ ]:
data = np.array(data).reshape((70, 130))
fig,axes = plt.subplots(1,1,figsize=(22,8))
axes.imshow(data, interpolation="bicubic", cmap='jet');
axes.grid(False)

El gráfico anterior tiene un problema. Los colores se asignan automáticamente, con el menor valor asignado a azul y los más altos a rojo (y el resto entre estos colores).

El README provisto indica que las temperaturas continentales están disponibles. Para el mar, se asigna el valor -999.99. 

El problema es que las temperaturas de interés (aproximadamente entre -25C y 40C) están comprimidas en el tope de escala. Una forma de evitar esto es normalizando los valores de interés.

In [ ]:
from matplotlib import colors
norm = colors.Normalize(
     vmin=data[data > -100.0].min(),
     vmax=data[data <100.0].max()
)

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(22,8))
axes.imshow(data, interpolation="bicubic", norm=norm, cmap='jet');
axes.grid(False)

El próximo paso es combinar esta imagen con una imagen de Europa.

In [ ]:
from matplotlib import image

def plot_temperatures(year, season):
    # Nota: no debería haber rutas hardcoded.
    data = get_temperatures(DATASET_PATH+"/TT_Europe_1500_2002_New.GDX",year = year,season = season)
    data = np.array(data).reshape((70, 130))
    
    europe_img = image.imread(DATASET_PATH+"/europe.png")
    
    norm = colors.Normalize(
         vmin=data[data > -100.0].min(),
         vmax=data[data <100.0].max()
    )
    
    fig,axes = plt.subplots(1,1,figsize=(20,8))
    axes.imshow(europe_img)
    im = axes.imshow(data,
         interpolation="bicubic",
         norm=norm,
         alpha=0.43,
         extent=(0,europe_img.shape[1],europe_img.shape[0],0),
         cmap='jet'
    )
    fig.suptitle("Temperaturas en Europa en {} de {}".format(season,year) )
    fig.colorbar(im)
    axes.grid(False)
    
    
plot_temperatures(1700, "Otoño")

Bonus: escribir una función que para un intervalo de años, grafique o anime las estaciones, por ejemplo:

<center>
<img src="assets/multiple_seasons.png"/>
</center>

## 5. Conclusiones

En este cuaderno se dieron los primeros pasos en el uso de herramientas para poder manipular archivos, trabajar con arrays y realizar visualizaciones.

En lo sucesivo se irán incorporando nuevas técnicas de análisis y procesamiento, con el objetivo de utilizarlas también en la preparación de datos para modelos de aprendizaje automático.

## Referencias

### Datos de temperaturas

Luterbacher, J., et al.  2006.  
European Seasonal Temperature Reconstructions.  
IGBP PAGES/World Data Center for Paleoclimatology  
Data Contribution Series # 2006-060.  
NOAA/NCDC Paleoclimatology Program, Boulder CO, USA.

### Grilla de temperatura absoluta del aire en superficie para Europa, años 1500-2002:

Luterbacher, J., Dietrich, D., Xoplaki, E., Grosjean, M., and Wanner, H., 2004:  
European seasonal and annual temperature variability, trends, and extremes since 1500,  
Science 303, 1499-1503 ([DOI:10.1126/science.1093877](http://doi.org/10.1126/science.1093877)).

Xoplaki, E., Luterbacher, J., Paeth, H., Dietrich, D., Steiner N., Grosjean, M., and Wanner, H., 2005:  
European spring and autumn temperature variability and change of extremes over the last half millennium,  
Geophys. Res. Lett., 32, L15713 ([DOI:10.1029/2005GL023424](http://doi.org/10.1029/2005GL023424)).

Nota: Este cuaderno fue adaptado de https://personalpages.manchester.ac.uk/staff/stefan.guettel/py/10a-data_analysis-2020.pdf
